In [719]:
import tarfile
from six.moves import urllib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

In [720]:
# Mi sembra più leggibile mettere questa riga in una casella diversa dagli import
data = pd.read_csv("Matriz_Completa.csv")

In [712]:
# NON abbiamo più bisogno di questa funzione, il suo compito è rimpiazzato da quella nella casella successiva
'''
def split_train_test(d, test_ratio):
    shuffled_indices = np.random.permutation(len(d))
    test_set_size = int(len(d)*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return d.iloc[train_indices], data.iloc[test_indices]

train_set, test_set = split_train_test(data, 0.2)
print(len(train_set), "train + ", len(test_set), "test")
'''

1424 train +  356 test


In [728]:
# La classe StratifiedShuffleSplit esegue uno split del dataset in train_set e test_set rispettando la
# distribuzione dei valori di tutto il dataset. In questo caso stiamo stratificando rispetto alla colonna TIPO.
# L'attributo test_size = 0.2 indica che il test_set comprende il 20% dei valori dell'intero dataset

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(data, data["TIPO"]):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]

In [729]:
# Qui abbiamo le percentuali di siti Benigni e Maligni nel dataset

data["TIPO"].value_counts()/len(data)

Benigna    0.878652
Maligna    0.121348
Name: TIPO, dtype: float64

In [730]:
# Qui abbiamo le dimensioni di test_set e train_set

print(len(strat_train_set), len(strat_test_set))

1424 356


In [732]:
# Come possiamo vedere, le percentuali sono state rispettate

strat_train_set["TIPO"].value_counts()/len(strat_train_set)

Benigna    0.878511
Maligna    0.121489
Name: TIPO, dtype: float64

In [733]:
def getValidIndex(invalidIndex, lenght):
    indici = np.array(range(lenght))
    valid = np.delete(indici, invalidIndex)
    return valid

In [734]:
class Column_selector(BaseEstimator, TransformerMixin):
    def __init__(self, col_indexes):
        self.col_indexes = col_indexes
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        return X[:, self.col_indexes].reshape(len(X[:, self.col_indexes]), len(self.col_indexes))
    
class StringUpper(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        r = [str(element).upper() for element in X]
        return np.array(r).reshape(-1, 1)
    
class ColumnLabelBinarizer(LabelBinarizer):
    def fit_transform(self,X,y=None):
        return super(ColumnLabelBinarizer,self).fit_transform(X)
    
class NumberCheckNan(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None 
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        for i in range (0, len(X)):
            for j in range(0, len(X[0])):
                if X[i][j] is None or X[i][j]!= X[i][j] or X[i][j] == "None":
                    X[i][j] = 0
        return np.array(X).reshape(-1, len(X[0]))
    
    
class CheckNan(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None 
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        for i in range (0, len(X)):
            if X[i]!= X[i]:
                X[i] = "None"
        return np.array(X).reshape(-1, 1)
    
class DataFrameToArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None  
    def fit(self, X, y= None):
        return self
    def transform(self, X):        
        return X.values

    
class ApacheTransform(BaseEstimator, TransformerMixin):
    #col_indexes Indici sui quali la pipeline deve operare
    def __init__(self):
        return None  
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        #print(X)
        #X è t1 passato come parametro in c= Server_Pipeline.fit_transform(t1)
        Y=np.array([])
        for i in range(0, len(X)):
            Y= np.append(Y,int("APACHE" in X[i]))
        return Y.reshape(-1, 1)

class CacheHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.value_list = {}
        return None
    def fit(self, X, y= None):
        X = X.tolist()
        for j in range (0, len(X)):
            value = X[j]
            if X[j] is None:
                cell_values="None"
            else:
                cell_values= "".join(X[j][0].split(" ")).split(";")
            for cell_value in cell_values:
                if cell_value not in self.value_list:
                    self.value_list[cell_value] = len(self.value_list.keys())
        return  self  
    def transform(self, X):
        matrix=[]
        for i in range (0, len(X)):
            cell_values=""
            vettoreIesimaCella= (np.zeros(len(self.value_list.keys()),dtype=int)).tolist()
            if X[i] is None:
                cell_values="None"
            else:
                cell_values= "".join(X[i][0].split(" ")).split(";")
            for cell_value in cell_values:
                if cell_value in self.value_list:
                    indexToChange=self.value_list[cell_value]
                    vettoreIesimaCella[indexToChange]=1 
            matrix.append(vettoreIesimaCella) 
        return np.array(matrix).reshape(-1, len(matrix[0]))
    
class extension_extractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        result = []
        for i in range(len(X)):
            substrings = X[i][0].split(".") 
            result.append(substrings[len(substrings)-1])
        return np.array(result)


In [735]:
prePipeline = Pipeline([
    ('dataFrameSelector', DataFrameToArray()),
    ('column_selector', Column_selector(col_indexes=getValidIndex([0,9,10], len(strat_train_set.columns))))
])

In [736]:
charsetPipeline = Pipeline([
    ('Column_selector', Column_selector(col_indexes=[2])),
    ('checkNan', CheckNan()),
    ('stringUpper', StringUpper()),
    ('columnLabelBinarizer',ColumnLabelBinarizer())
])

In [737]:
Server_Apache_Pipeline = Pipeline([
     ("column_selector", Column_selector(col_indexes=[3])),
     ("CheckNan", CheckNan()),
     ("StringUpper", StringUpper()),
     ("ApacheTransform", ApacheTransform())
   ])

In [738]:
Server_Pipeline = Pipeline([
    ("column_selector", Column_selector(col_indexes=[3])),
     #Effettuo l'UPPERCASE della colonna selezionata e la passo al transform successivo
    ("check_nan", CheckNan()),
    ("StringUpper", StringUpper()),
    #Prendo l'UPPERCASE e trasformo tutto in binario
    ("ColumnLabelBinarizer", ColumnLabelBinarizer())
])

In [739]:
server_feature_union = FeatureUnion([
    ("server_pipeline_1", Server_Apache_Pipeline),
    ("Server_Pipeline", Server_Pipeline)
])

In [740]:
Cache_Pipeline = Pipeline([
    ("column_selector", Column_selector(col_indexes=[4])),
    ("StringUpper", StringUpper()), 
    ("CacheHandler", CacheHandler())
])

In [741]:
content_length_pipeline = Pipeline([
    ('column_selector', Column_selector(col_indexes = [5])),
    ('imputer', Imputer(strategy = "median")),
    ('std_scaler', StandardScaler())
])

In [742]:
countryPipeline = Pipeline([
    ('column_selector', Column_selector(col_indexes=[6])),
    ('checkNan', CheckNan()),
    ('stringUpper', StringUpper()),
    ('columnLabelBinarizer',ColumnLabelBinarizer())
])

In [743]:
provincePipeline = Pipeline([
    ('column_selector', Column_selector(col_indexes=[7])),
    ('checkNan', CheckNan()),
    ('stringUpper', StringUpper()),
    ('columnLabelBinarizer',ColumnLabelBinarizer())
])

In [744]:
portPipeline = Pipeline([
    ('column_selector', Column_selector(col_indexes=[10])),
    ('checkNan', CheckNan()),
    ('stringUpper', StringUpper()),
    ('columnLabelBinarizer',ColumnLabelBinarizer())  
])

In [745]:
domain_pipeline = Pipeline([
    ("column_selector", Column_selector([8])),
    ("extension_extractor", extension_extractor()),
    ("label_binarizer", ColumnLabelBinarizer())
])

In [746]:
select_and_scale1 = Pipeline([
    ("column_selector", Column_selector([0, 1])),
    ("checkNumberNan", NumberCheckNan()),
    ('std_scaler', StandardScaler())
])

In [747]:
select_and_scale2 = Pipeline([
    ("column_selector", Column_selector([9])),
    ("checkNumberNan", NumberCheckNan()),
    ('std_scaler', StandardScaler())
])

In [748]:
select_and_scale3 = Pipeline([
    ("column_selector", Column_selector([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])),
    ("checkNumberNan", NumberCheckNan()),
    ('std_scaler', StandardScaler())
])

In [749]:
full_pipeline = FeatureUnion([
    #("add_url_length", Column_selector([0])),
    #("add_num_special_charater", Column_selector([1])),
    ('select_and_scale1', select_and_scale1),
    ("charset_pipeline", charsetPipeline),
    ("server_pipeline", server_feature_union),
    ("cache_pipeline", Cache_Pipeline),
    ("content_length_pipeline", content_length_pipeline),
    ("countryPipeline", countryPipeline),
    ("provincePipeline", provincePipeline),
    ("domain_pipeline", domain_pipeline),
    #("add_col_9", Column_selector([9])),
    ('select_and_scale2', select_and_scale2),
    ("portPipeline", portPipeline),
    ('select_and_scale3', select_and_scale3)
    #("add_remaining_cols_", Column_selector([11, 12, 13, 14, 15, 16, 17, 18, 19, 20]))
])

In [752]:
# Definisco una pipeline che mette insieme la prePipeline e la full_pipeline, in modo da non dover fare due invocazioni

data_preparator = Pipeline([
    ("prePipeline", prePipeline),
    ("full_pipeline", full_pipeline)
])

In [753]:
# Preparo i dati per gli algoritmi

train_set_prepared = data_preparator.fit_transform(strat_train_set)

/Users/giovanni/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/giovanni/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
/Users/giovanni/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:42: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
/Users/giovanni/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/giovanni/anaconda/lib/python3

In [754]:
# Definisco un label_binarizer, questo servirà a trasformare in binario le etichette 'Benigno'/'Maligno'

label_binarizer = LabelBinarizer()

In [755]:
# Qui possiamo vedere la struttura del train_set trasformato

train_set_prepared.shape

(1424, 645)

In [802]:
# Otteniamo le etichette binarizzate per il train_set. Notare che qui viene invocato il metodo .fit_transform() !!

train_set_labels = label_binarizer.fit_transform(strat_train_set["TIPO"])

# Usiamo la funzione flatten per modificare la formattazione di serie del vettore che è
# del tipo v = [[e1], [e2], [e3], ..., [en]] in v = [e1, e2, e3, ..., en]

train_set_labels = train_set_labels.flatten()

In [803]:
# Definiamo una funzione che misura l'accuratezza delle predizioni dell'algoritmo

def accuracy_rate(labels, predictions):
    count = 0
    for i in range(len(labels)):
        if labels[i] == predictions[i]:
            count += 1
    return count/len(labels)

In [804]:
# Importiamo il modello SVC. Esso è un Classificatore binario (riesce a classificare tra due sole classi) basato
# sull'algoritmo SVM = Support Vector Machines.

from sklearn.svm import LinearSVC

In [805]:
# Facciamo un test preventivo per misurare l'accuratezza dell'algoritmo, lasciando i parametri di default

classifier = LinearSVC(C= 1, loss = "hinge")

In [806]:
# Eseguiamo il fit (= allenamento) dell'algoritmo sul train_set

classifier.fit(train_set_prepared, train_set_labels)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [807]:
# Misuriamo l'accuratezza del classificatore sul train_set, ovvero sui dati sul quale è stato allenato

predictions_train = classifier.predict(train_set_prepared)

In [808]:
# Misuriamo l'accuratezza delle predizioni

accuracy_rate(train_set_labels, predictions_train)

0.9859550561797753

In [809]:
# L'accuratezza sembra eccessiva, facendo pensare che l'algoritmo sia andato in overfitting. Per regolarizzare
# le predizioni, usiamo la classe GridSearchCV, la quale divide il train_set in tante coppie di 
# mini_train_set/mini_test_set ed allena l'algoritmo con vari parametri, restituendoci quelli che
# hanno performato meglio

from sklearn.model_selection import GridSearchCV

In [810]:
# indichiamo i valori del parametri su cui vogliamo testare l'algoritmo

param_grid = [
    {'C': [.1, .5, 1, 10, 50, 100, 500]}, 
    {"loss": ["hinge", "squared_hinge"]}
]

In [811]:
# Inizializziamo il classificatore

classifier = LinearSVC()

In [814]:
# Alleniamo l'algoritmo nel modo prestabilito. Il parametri cv indica il numero di split del train_set in
# 'mini_train_set'/'mini_test_set', e quindi le dimensioni degli stessi.

grid_search = GridSearchCV(classifier, param_grid, cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(train_set_prepared, train_set_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.1, 0.5, 1, 10, 50, 100, 500]}, {'loss': ['hinge', 'squared_hinge']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [816]:
# Chiediamo semplicemente a grid_search qual è la migliore combinazione possibile di parametri: il fatto che
# la misura dell'errore non sia specificata ci indica che è indifferente ai fini delle performances

grid_search.best_params_

{'C': 1}

In [823]:
#Facciamo una ricerca più 'raffinata' per il parametro 'C': il risultato ci indica che C=1 ci assicura
#La maggiore accuratezza

grid_search = GridSearchCV(classifier, param_grid= [{'C': [7, 8.5, 1, 11.5, 13]}], cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(train_set_prepared, train_set_labels)
grid_search.best_params_

{'C': 1}

In [827]:
#Siamo pronti quindi per misurare le performances sul test_set! Innanzitutto lo alleniamo con i parametri migliori, e
#ne misuriamo nuovamente l'accuratezza sul train_set

classifier = LinearSVC(C= 1, loss = "hinge")
classifier.fit(train_set_prepared, train_set_labels)
predizioni = classifier.predict(train_set_prepared)
accuracy_rate(train_set_labels, predizioni)

0.9859550561797753

In [828]:
# A questo punto calcoliamo le predizioni sul test_set (dati che l'algoritmo non ha mai visto), e ne misuriamo 
# l'accuratezza. 

# Innanzitutto prepariamo i dati e calcoliamo le etichette (notare che in questo caso vengono chiamati i metodi
# .transform() e non fit_transform() !! )

test_set_prepared = data_preparator.transform(strat_test_set)
test_set_labels = label_binarizer.transform(strat_test_set["TIPO"])
predizioni_test = classifier.predict(test_set_prepared)

accuracy_rate(test_set_labels, predizioni_test)

/Users/giovanni/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


0.9859550561797753

In [829]:
# Possiamo vedere che l'accuratezza sul test_set (= dati che l'algoritmo non ha mai visto) raggiunge il 98.6%

In [830]:
# Alleniamo adesso un classificatore basato su reti neurali trmite tensorflow
import tensorflow as tf

In [831]:
print(tf.__version__)

1.1.0


In [885]:
# Creiamo un oggetto iterabile sulle colonne del test_set

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_set_prepared)

In [891]:
# Creiamo una istanza di DNNClassifier, un classificatore basato su reti neurali profonde. In questo caso abbiamo tre
#'livelli nascosti', che si compongono rispettivamente di 100, 70 e 50 neuroni. Abbiamo scelto di usare
# una rete a tre livelli (ma con un numero moderato di neuroni) piuttosto che avere 1-2 livelli ma più ampi,
# poiché aumentare la profondità della rete assicura performances migliori rispetto ad aumentarne l'ampiezza.

dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[100, 70, 50], n_classes = 2, feature_columns = feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12db5bf98>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


In [892]:
# Questo cast è necessario per le versioni più recenti di TesorFlow
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)

In [896]:
# Eseguiamo il training dell'algoritmo. Con questi parametri l'algoritmo ripete 500 iterazioni del training e,
# ad ogni iterazione, considera 40 elementi del train_set (scelti uniformemente a caso) 

dnn_clf.fit(train_set_prepared, train_set_labels, batch_size = 40, steps=300)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/1g/gcgbjxpj23g6k44tq1_24zv80000gp/T/tmpykfjn3d0/model.ckpt-250
INFO:tensorflow:Saving checkpoints for 251 into /var/folders/1g/gcgbjxpj23g6k44tq1_24zv80000gp/T/tmpykfjn3d0/model.ckpt.
INFO:tensorflow:loss = 0.0333497, step = 251
INFO:tensorflow:global_step/sec: 109.906
INFO:tensorflow:loss = 0.0433373, step = 351 (0.977 sec)
INFO:tensorflow:global_step/sec: 138.898
INFO:tensorflow:loss = 0.0457218, step = 451 (0.661 sec)
INFO:tensorflow:Saving checkpoints for 550 into /var/folders/1g/gcgbjxpj23g6k44tq1_24zv80000gp/T/tmpykfjn3d0/model.ckpt.
INFO:tensorflow:Loss for f

SKCompat()

In [897]:
# Calcoliamo le predizioni riguardo al train_set

predizioni_train = dnn_clf.predict(train_set_prepared)

INFO:tensorflow:Restoring parameters from /var/folders/1g/gcgbjxpj23g6k44tq1_24zv80000gp/T/tmpykfjn3d0/model.ckpt-550


In [899]:
# Calcoliamo l'accuratezza delle precisioni

accuracy_rate(train_set_labels, predizioni_train["classes"])

0.9985955056179775

In [909]:
# L'accuratezza sul test_set risulta molto alta: ciò può essere dovuto sia alla eccessiva complessità del modello,
# che può aver appreso anche il 'rumore' presente sui dati, andando in overfitting, sia al fatto che il modello
# risulti effettivamente accurato. Pur essendo l'accuratezza così alta, essa potrebbe essere comunque indice di
# un modello accurato siccome questo numero non si discosta tanto dalla accuratezza ottenuta tramite
# il classificatore SVM. Perciò, ci limiteremo a controllare se il modello è affidabile o meno, semplicemente
# controllando l'accuratezza sul test_set.

predicted_test = dnn_clf.predict(test_set_prepared)

INFO:tensorflow:Restoring parameters from /var/folders/1g/gcgbjxpj23g6k44tq1_24zv80000gp/T/tmpykfjn3d0/model.ckpt-550


In [910]:
accuracy_rate(test_set_labels.flatten(), predicted_test["classes"])

0.9971910112359551

In [924]:
# Proviamo inoltre ad allenare un ultimo modello, detto RandomForestClassifier. Esso è basato sull'algoritmo 
# DecisionTree, e consiste nel creare n_estimators Decision Trees, ognuna con un numero massimo di nodi
# uguale a max_leaf_nodes. In questo primo caso utilizziamo dei parametri di esempio, con lo scopo di affinarli
# successivamente

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 100, max_leaf_nodes = 13)

In [925]:
# Alleniamo l'algoritmo tramite il metodo .fit()

rnd_clf.fit(train_set_prepared, train_set_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=13,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [928]:
#Misuriamo l'accuratezza delle predizioni sui dati sui quali l'algoritmo è stato allenato.

predizioni_train = rnd_clf.predict(train_set_prepared)
accuracy_rate(predizioni_train, train_set_labels)

0.9367977528089888

In [931]:
# Utilizziamo la classe grid_search per fare diversi test sugli iperparametri dell'algoritmo

param_grid = [
    {"max_leaf_nodes": [5, 8, 10, 12, 15]},
    {'n_estimators': [50, 75, 100, 175, 250, 500]}
]
grid_search = GridSearchCV(rnd_clf, param_grid, cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(train_set_prepared, train_set_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=13,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_leaf_nodes': [5, 8, 10, 12, 15]}, {'n_estimators': [50, 75, 100, 175, 250, 500]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [933]:
# Chiediamo quindi a grid_search quali siano i parametri migliori: la risposta è {'max_leaf_nodes': 15}, ad indicare
# che il parametro n_estimators non impatta in modo significativo l'accuratezza

grid_search.best_params_

{'max_leaf_nodes': 15}

In [936]:
# Eseguiamo quindi una ricerca più mirata sul parametro max_leaf_nodes

param_grid = [
    {"max_leaf_nodes": [13, 14, 15, 16, 17, 18, 19]},
    {'n_estimators': [50]}
]
grid_search = GridSearchCV(rnd_clf, param_grid, cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(train_set_prepared, train_set_labels)
grid_search.best_params_

{'max_leaf_nodes': 19}

In [946]:
# Ancora...

param_grid = [
    {"max_leaf_nodes": [19, 20, 21, 22, 23, 24, 25, 26]},
    {'n_estimators': [50]}
]
grid_search = GridSearchCV(rnd_clf, param_grid, cv = 5, scoring = 'neg_mean_squared_error')
grid_search.fit(train_set_prepared, train_set_labels)
grid_search.best_params_

{'max_leaf_nodes': 25}

In [947]:
# eseguiamo un training dell'algoritmo con i parametri indicati da grid_search, e ne misuriamo l'accuratezza

rnd_clf = RandomForestClassifier(n_estimators = 50, max_leaf_nodes = 24)
rnd_clf.fit(train_set_prepared, train_set_labels)
predizioni_train = rnd_clf.predict(train_set_prepared)
accuracy_rate(predizioni_train, train_set_labels)

0.9550561797752809

In [948]:
# Testiamo l'algoritmo sul test_set

predizioni_test = rnd_clf.predict(test_set_prepared)
accuracy_rate(predizioni_test, test_set_labels)

0.9466292134831461

In [ ]:
# Proviamo a mettere insieme tutti e tre i classificatori costruiti fino ad ora tramite la classe
# BaggingClassifier. 